# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [1]:
...

Ellipsis

In [2]:
import random
from collections import defaultdict, Counter
import numpy as np


def preprocess_text(text):
    """Добавляет тэги <start> и токенизирует текст."""
    sentences = text.lower().split('.')
    tokenized_sentences = [['<start>', '<start>'] + sent.strip().split() + ['<end>'] for sent in sentences if sent]
    return tokenized_sentences

def build_ngram_counts(sentences, n=3):
    """Собирает статистику для n-грамм, биграмм и униграмм."""
    unigrams = Counter()
    bigrams = Counter()
    trigrams = Counter()

    for sentence in sentences:
        for i in range(len(sentence)):
            unigrams[sentence[i]] += 1
            if i >= 1:
                bigrams[(sentence[i-1], sentence[i])] += 1
            if i >= 2:
                trigrams[(sentence[i-2], sentence[i-1], sentence[i])] += 1

    return unigrams, bigrams, trigrams

def calculate_trigram_probabilities(unigrams, bigrams, trigrams, vocab_size):
    """Рассчитывает вероятности P(word | word1, word2) с учетом сглаживания."""
    probabilities = defaultdict(float)

    for (word1, word2, word3), trigram_count in trigrams.items():
        bigram_count = bigrams[(word1, word2)]
        probabilities[(word1, word2, word3)] = (trigram_count + 1) / (bigram_count + vocab_size)

    return probabilities

def generate_text(trigram_probs, unigrams, n=20):
    """Генерирует текст с использованием треграммной модели."""
    text = ['<start>', '<start>']
    vocab = list(unigrams.keys())

    for _ in range(n):
        prev_bigram = (text[-2], text[-1])
        candidates = {word3: prob for (word1, word2, word3), prob in trigram_probs.items() if (word1, word2) == prev_bigram}
        if candidates:
            next_word = random.choices(list(candidates.keys()), weights=list(candidates.values()))[0]
        else:
            next_word = random.choice(vocab)
        if next_word == '<end>':
            break
        text.append(next_word)

    return ' '.join(text[2:])

def calculate_perplexity(test_sentences, trigram_probs, bigrams, vocab_size):
    """Вычисляет перплексию для тестового множества."""
    log_prob_sum = 0
    total_words = 0

    for sentence in test_sentences:
        for i in range(2, len(sentence)):
            trigram = (sentence[i-2], sentence[i-1], sentence[i])
            bigram = (sentence[i-2], sentence[i-1])
            trigram_prob = trigram_probs.get(trigram, 1 / (bigrams[bigram] + vocab_size))
            log_prob_sum += -np.log2(trigram_prob)
            total_words += 1

    return 2 ** (log_prob_sum / total_words)

In [3]:
# Текстовый корпус
text = """
I love natural language processing. It is fascinating to work with text data.
Language modeling is an essential part of NLP. Deep learning models are also great for text analytics.
"""

sentences = preprocess_text(text)

unigrams, bigrams, trigrams = build_ngram_counts(sentences)
vocab_size = len(unigrams)

trigram_probs = calculate_trigram_probabilities(unigrams, bigrams, trigrams, vocab_size)

# Генерация текста
for _ in range(5):
    print("Generated text:", generate_text(trigram_probs, unigrams))

# Тестовое множество для перплексии
test_sentences = preprocess_text("Text analysis is fun. I enjoy working on NLP tasks.")
perplexity = calculate_perplexity(test_sentences, trigram_probs, bigrams, vocab_size)
print("Perplexity:", perplexity)

Generated text: language modeling is an essential part of nlp
Generated text: language modeling is an essential part of nlp
Generated text: deep learning models are also great for text analytics
Generated text: language modeling is an essential part of nlp
Generated text: i love natural language processing
Perplexity: 28.187242351121117


## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [4]:
def generate_with_beam_search(trigram_probs, unigrams, prompt=None, beam_width=3, max_len=20):
    """
    Генерация текста с использованием бим-сьёрча (beam search) для треграммной языковой модели.

    Параметры:
        trigram_probs (dict): вероятности триграмм P(word | word1, word2).
        unigrams (Counter): частоты униграмм для выбора случайных слов.
        prompt (list or None): начальный промпт для генерации (например, ['<start>', '<start>']).
        beam_width (int): ширина луча beam search.
        max_len (int): максимальная длина генерируемого текста.

    Возвращает:
        generated_text (str): сгенерированный текст.
    """
    if prompt is None:
        prompt = ['<start>', '<start>']  # Если промпт не задан, начинаем с тэгов <start>
    else:
        # Если промпт короче двух слов, добавляем <start>
        prompt = ['<start>'] * (2 - len(prompt)) + prompt

    # Начальные состояния луча
    sequences = [(prompt, 0)]  # пары (последовательность, логарифмическая вероятность)

    for _ in range(max_len):
        all_candidates = []

        # Для каждой последовательности в текущем биме расширяем возможные слова
        for seq, score in sequences:
            prev_bigram = (seq[-2], seq[-1])  # Два последних слова из последовательности
            candidates = {word3: prob for (word1, word2, word3), prob in trigram_probs.items() if (word1, word2) == prev_bigram}

            # Если нет возможных кандидатов (не встречается такой биграмма), случайный выбор
            if not candidates:
                candidates = {random.choice(list(unigrams.keys())): 1 / len(unigrams)}

            # Для каждого кандидата создаем новый путь
            for word, prob in candidates.items():
                new_seq = seq + [word]  # Расширяем последовательность
                new_score = score - np.log2(prob)  # Логарифмическая сумма вероятностей (минимизация отриц. лог)
                all_candidates.append((new_seq, new_score))

        # Сортируем кандидатов по вероятности (первым идут лучшие) и берем top-K
        sequences = sorted(all_candidates, key=lambda x: x[1])[:beam_width]

        # Если все последовательности заканчиваются на <end>, выходим
        if all(seq[-1] == '<end>' for seq, _ in sequences):
            break

    # Возвращаем последовательность с наивысшей вероятностью
    best_sequence = min(sequences, key=lambda x: x[1])[0]

    # Убираем стартовые токены и токен <end>
    return ' '.join(word for word in best_sequence if word not in ['<start>', '<end>'])

text = """
I love natural language processing. It is fascinating to work with text data.
Language modeling is an essential part of NLP. Deep learning models are also great for text analytics.
"""
sentences = preprocess_text(text)
unigrams, bigrams, trigrams = build_ngram_counts(sentences)
vocab_size = len(unigrams)
trigram_probs = calculate_trigram_probabilities(unigrams, bigrams, trigrams, vocab_size)

# Генерация текста с bим-сьёрч
print("Generated with default start:")
print(generate_with_beam_search(trigram_probs, unigrams, prompt=['<start>', '<start>']))

print("\nGenerated with prompt 'language is':")
print(generate_with_beam_search(trigram_probs, unigrams, prompt=['language', 'is']))

print("\nGenerated with prompt 'deep learning':")
print(generate_with_beam_search(trigram_probs, unigrams, prompt=['deep', 'learning']))


Generated with default start:
it is fascinating to work with text data language work nlp fascinating nlp i fascinating text processing processing natural

Generated with prompt 'language is':
language is to essential for models love an for fascinating language natural is is fascinating to work with text data

Generated with prompt 'deep learning':
deep learning models are also great for text analytics
